In [6]:
# df = pd.read_csv('Convolutional-Neural-Stock-Market-Technical-Analyser/data/aapl/AAPL.csv', sep = ',' \
#                  )
# dataset = df['Adj Close'].values
# df.head()

In [7]:
ls

LSTM_stock01.ipynb*
README.md*
ScrapYahooKeyStat.ipynb*
Stock_Prediction_General_Method.py*
Stock_Prediction_Global_Parameters.py*
Stock_Prediction_Stateless_LSTM_Model.py*
Stock_Prediction_Stateless_LSTM_Model_01.py*
Stock_Prediction_Stateless_LSTM_Model_02.py*
Stock_Prediction_Stateless_LSTM_Run.ipynb*
Stock_Prediction_Stateless_LSTM_Run.py*
Untitled.ipynb
__pycache__/
history/
multiple_input_features.ipynb*
stock_model.ipynb*
stock_prediction .ipynb
time-series_README.pdf


In [1]:
# LSTM for international airline passengers problem with regression framing
import numpy as np
import matplotlib.pyplot as plt
#from pandas import read_csv
import pandas as pd 
import math
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error


Using TensorFlow backend.


In [2]:
# convert an array of values into a dataset matrix
def create_dataset(dataset, look_back=1):
	dataX, dataY = [], []
	for i in range(len(dataset)-look_back-1):
		a = dataset[i:(i+look_back), :]
		dataX.append(a)
		dataY.append(dataset[i + look_back, 0])
	return np.array(dataX), np.array(dataY)


In [18]:
df = pd.read_csv('../Convolutional-Neural-Stock-Market-Technical-Analyser/data/aapl/AAPL.csv', names = ['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume'], sep = ',')


In [25]:
df['High'].astype('float')

ValueError: could not convert string to float: 'High'

In [24]:
# fix random seed for reproducibility
np.random.seed(7)
# load the dataset
# df = pd.read_csv('../daily/table_aapl.csv', sep = ',')
df['diff'] = df['High'].astype('float') - df['Low'].astype('float')
#df01 = df[['Adj Close', 'Open', 'Volume', 'diff']]
dataset = np.array([df['Adj Close'].values, df['Open'].values, df['Volume'].values, df['diff'].values])
dataset = dataset.astype('float32')
dataset = dataset.reshape(dataset.shape[1], dataset.shape[0])
#date = df['Date'].values
#date = date.astype('float32')
#date = date.reshape(-1, 1)
#type(dataset)
#df01.head()
#dataset.shape[1]
df.head()
dataset.shape

ValueError: could not convert string to float: 'High'

In [4]:
# %matplotlib inline 

# #df['Volume'].plot(kind = 'line')
# import matplotlib.pyplot as plt 

# total_value = df['Adj Close']*df['Volume']
# plt.plot(total_value)

In [5]:
# normalize the dataset
from sklearn import preprocessing

scaler = MinMaxScaler(feature_range=(0, 1))
dataset = scaler.fit_transform(dataset)

dataset.shape
dataset

NameError: name 'dataset' is not defined

In [ ]:
# split into train and test sets
train_size = int(len(dataset) * 0.67)
test_size = len(dataset) - train_size
train, test = dataset[0:train_size,:], dataset[train_size:len(dataset),:]
train

In [ ]:
# from sklearn import preprocessing

# scaler = MinMaxScaler(feature_range=(0, 1))

# train = dataset[0:train_size,:]


In [ ]:
# reshape into X=t and Y=t+1
look_back = 5
trainX, trainY = create_dataset(train, look_back)
testX, testY = create_dataset(test, look_back)
trainY

In [ ]:
trainX.shape

In [ ]:
# reshape input to be [samples, time steps, features]
trainX = np.reshape(trainX, (trainX.shape[0], trainX.shape[1], trainX.shape[2]))
testX = np.reshape(testX, (testX.shape[0], testX.shape[1], testX.shape[2]))
trainX.shape

In [ ]:
print(trainX.shape)
print(trainY.shape)

In [ ]:
# create and fit the LSTM network
from keras import regularizers


model = Sequential()
model.add(LSTM(128, input_shape=(5, 4), kernel_regularizer=regularizers.l2(1),
                activity_regularizer=regularizers.l1(0.01)))
model.add(Dense(1))



model.compile(loss='mean_absolute_percentage_error', optimizer='adam')
model.fit(trainX, trainY, epochs=2, batch_size=1000, verbose=2)
# make predictions
trainPredict = model.predict(trainX)
testPredict = model.predict(testX)


In [ ]:
testPredict.shape

In [ ]:
# invert predictions
trainPredict = scaler.inverse_transform(trainPredict)
trainY = scaler.inverse_transform([trainY])
testPredict = scaler.inverse_transform(testPredict)
testY = scaler.inverse_transform([testY])
# calculate root mean squared error
trainScore = math.sqrt(mean_squared_error(trainY[0], trainPredict[:,0]))
print('Train Score: %.2f RMSE' % (trainScore))
testScore = math.sqrt(mean_squared_error(testY[0], testPredict[:,0]))
print('Test Score: %.2f RMSE' % (testScore))


In [ ]:
# # invert predictions
# trainPredict = scaler.inverse_transform(trainPredict)




In [ ]:
# shift train predictions for plotting

plt.figure(figsize = (15, 5))
trainPredictPlot = np.empty_like(dataset)
trainPredictPlot[:, :] = np.nan
trainPredictPlot[look_back:len(trainPredict)+look_back, :] = trainPredict
# shift test predictions for plotting
testPredictPlot = np.empty_like(dataset)
testPredictPlot[:, :] = np.nan
testPredictPlot[len(trainPredict)+(look_back*2)+1:len(dataset)-1, :] = testPredict
# plot baseline and predictions
plt.plot(scaler.inverse_transform(dataset))
plt.plot(trainPredictPlot)
plt.plot(testPredictPlot)

plt.show()

In [ ]:
# shift train predictions for plotting
trainPredictPlot = np.empty_like(dataset)
trainPredictPlot[:, :] = np.nan
trainPredictPlot[look_back:len(trainPredict)+look_back, :] = trainPredict
# shift test predictions for plotting
testPredictPlot = np.empty_like(dataset)
testPredictPlot[:, :] = np.nan
testPredictPlot[len(trainPredict)+(look_back*2)+1:len(dataset)-1, :] = testPredict
# plot baseline and predictions
date = pd.to_datetime(df.Date)
plt.figure(figsize = (20, 5))
plt.plot(date, scaler.inverse_transform(dataset), color = 'r')
plt.plot(date[:len(trainPredictPlot)+1], trainPredictPlot, color = 'g')
plt.plot(date[:len(testPredictPlot)+1], testPredictPlot, color = 'b')
plt.xlim(('2017-08-01', '2017-09-28'))
plt.ylim((150, 165))
plt.show()

In [ ]:
len(date)